# 에어비앤비 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import urllib

In [2]:
url="https://www.airbnb.co.kr/s/%EC%A0%9C%EC%A3%BC%EB%8F%84/homes?query=%EC%A0%9C%EC%A3%BC%EB%8F%84&federated_search_session_id=e296e38b-6ec1-4296-9a81-f762cfb2c92a&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&checkin=2020-08-16&refinement_paths%5B%5D=%2Fhomes&checkout=2020-08-17&place_id=ChIJRUDITFTjDDURMb8emNI2vGY&section_offset=2&items_offset=0"
url_parsed=urllib.parse.urlparse(url)

url_qs=urllib.parse.parse_qs(url_parsed.query)

In [3]:
url_qs

{'query': ['제주도'],
 'federated_search_session_id': ['e296e38b-6ec1-4296-9a81-f762cfb2c92a'],
 'source': ['structured_search_input_header'],
 'search_type': ['pagination'],
 'tab_id': ['home_tab'],
 'checkin': ['2020-08-16'],
 'refinement_paths[]': ['/homes'],
 'checkout': ['2020-08-17'],
 'place_id': ['ChIJRUDITFTjDDURMb8emNI2vGY'],
 'section_offset': ['2'],
 'items_offset': ['0']}

In [4]:
base_url="https://www.airbnb.co.kr/s/%EC%A0%9C%EC%A3%BC%EB%8F%84/homes"
params={'query': '제주도',
 'federated_search_session_id': 'e296e38b-6ec1-4296-9a81-f762cfb2c92a',
 'source': 'structured_search_input_header',
 'search_type': 'pagination',
 'tab_id': 'home_tab',
 'checkin': '2020-08-16',
 'refinement_paths[]': '/homes',
 'checkout': '2020-08-17',
 'place_id': 'ChIJRUDITFTjDDURMb8emNI2vGY',
 'section_offset': '2',
 'items_offset': '0'}

In [5]:
resp=requests.get(base_url,params=params)
resp

soup=bs(resp.text)

item_tags=soup.select("#FMP-target > div > div > div > div")
len(item_tags[0])

<Response [200]>

In [8]:
a_tags=item_tags[0].find_all("a")

url_list=[]
for i in a_tags:
    url_list.append("https://www.airbnb.co.kr%s" %i.get("href"))
    
url_list


['https://www.airbnb.co.kr/rooms/19724369?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9',
 'https://www.airbnb.co.kr/rooms/36090662?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9',
 'https://www.airbnb.co.kr/rooms/15321610?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9',
 'https://www.airbnb.co.kr/rooms/18103390?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9',
 'https://www.airbnb.co.kr/rooms/43298030?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&fede

In [9]:
price_tags=item_tags[0].find_all("span", class_="_1p7iugi")

price_list=[]

for i in price_tags:
    try:
        price=i.text.split(":")[2]
    except:
        price=i.text.split(":")[1]
        
    price_list.append(price)
    
price_list
                  

['₩44,100',
 '₩51,988',
 '₩34,543',
 '₩35,814',
 '₩34,659',
 '₩80,870',
 '₩67,654',
 '₩103,977',
 '₩57,765',
 '₩46,212',
 '₩34,659',
 '₩34,659',
 '₩144,412',
 '₩83,181',
 '₩43,900',
 '₩92,423',
 '₩288,823',
 '₩19,640',
 '₩39,280',
 '₩25,417']

In [10]:
count_reviews=item_tags[0].find_all("span",class_="_krjbj")

count_reviews

count_reviews_list=[]
for i in count_reviews:
    if "후기" in str(i):
        count_reviews_list.append(i.text.split(" ")[1])
        
count_reviews_list

['702개',
 '36개',
 '63개',
 '70개',
 '10개',
 '64개',
 '44개',
 '317개',
 '8개',
 '15개',
 '54개',
 '188개',
 '96개',
 '156개',
 '7개',
 '121개',
 '57개',
 '14개']

In [11]:
name_tags=item_tags[0].find_all("div",class_="_1c2n35az")
rate_tags=item_tags[0].find_all("span",class_="_10fy1f8")


result=[]
for n,r,u,p,c in zip(name_tags,rate_tags,url_list,price_list,count_reviews_list):
    result.append({
        "이름":n.text,
        "평점":r.text,
        "후기 개수" :c,
        "가격":p,
        "링크":u
    })
    
result

[{'이름': '[플레이스캠프제주] Standard',
  '평점': '4.47',
  '후기 개수': '702개',
  '가격': '₩44,100',
  '링크': 'https://www.airbnb.co.kr/rooms/19724369?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9'},
 {'이름': '법환동 바닷가집 별채',
  '평점': '4.86',
  '후기 개수': '36개',
  '가격': '₩51,988',
  '링크': 'https://www.airbnb.co.kr/rooms/36090662?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9'},
 {'이름': '몽돌하우스',
  '평점': '4.84',
  '후기 개수': '63개',
  '가격': '₩34,543',
  '링크': 'https://www.airbnb.co.kr/rooms/15321610?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=d8ab127c-7d64-4fce-a573-0fdc5452e4c9'},
 {'이름': '걷기 좋은 동네 게스트하우스',
  '평점': '4.61',
  '후기 개수': '70개',
  '가격': '₩35,814',
  '링크': 'https://www.airbnb.co.k

# 에어비앤비 크롤링 함수화

In [255]:
def airbnb(place,checkin,checkout,pages):  #place = 장소, checkin = 체크인 날짜, checkout :체크아웃 날짜, pages=검색하고자 하는 페이지 수
    
    
    result_list=[]  #크롤링 최종 결과를 담을 리스트
    
    #에어비앤비는 offset이 20단위로 페이지가 변화 ex)offset 0 : 1페이지 , offset 1~20: 2페이지 
    for page in range(1,pages+1): #입력받은 페이지 수만큼 모두 크롤링 하기 위해
        if page==1:  #page가 1이면 offset=0
            offset=0
        else:
            offset=page*20  #1보다 크면 page*20
    
    
        #리퀘스트할 url과 파라미터 설정
        base_url="https://www.airbnb.co.kr/s/%EC%A0%9C%EC%A3%BC%EB%8F%84/homes" 
        params={'query': place,
         'federated_search_session_id': 'e296e38b-6ec1-4296-9a81-f762cfb2c92a',
         'source': 'structured_search_input_header',
         'search_type': 'pagination',
         'tab_id': 'home_tab',
         'checkin': checkin,
         'refinement_paths[]': '/homes',
         'checkout': checkout,
         'place_id': 'ChIJRUDITFTjDDURMb8emNI2vGY',
         'section_offset': '2',
         'items_offset': offset}
        
        resp=requests.get(base_url,params=params)
        soup=bs(resp.text)
        item_tags=soup.select("#FMP-target > div > div > div > div")
        len(item_tags[0])

        
        #해당 숙소에 대한 상세페이지 url을 가져옴
        a_tags=item_tags[0].find_all("a")
        url_list=[]
        for i in a_tags:
            url_list.append("https://www.airbnb.co.kr%s" %i.get("href"))


        #숙소들의 가격정보를 가져옴
        price_tags=item_tags[0].find_all("span", class_="_1p7iugi")
        price_list=[]
        for i in price_tags:
            try:
                price=i.text.split(":")[2]
            except:
                price=i.text.split(":")[1]

            price_list.append(price.replace("₩",""))

        #숙소들의 리뷰 개수를 가져옴
        count_reviews=item_tags[0].find_all("span",class_="_krjbj")
        count_reviews
        count_reviews_list=[]
        for i in count_reviews:
            if "후기" in str(i):
                count_reviews_list.append(i.text.split(" ")[1])

        #숙소들의 이름과 평점을 가져옴
        name_tags=item_tags[0].find_all("div",class_="_1c2n35az")
        rate_tags=item_tags[0].find_all("span",class_="_10fy1f8")
        
        result=[]
        
        #앞에서 추출한 정보들을 딕셔너리화 하여 result 리스트에 추가
        for n,r,u,p,c in zip(name_tags,rate_tags,url_list,price_list,count_reviews_list):
            result.append({
                "이름":n.text,
                "평점":r.text,
                "후기 개수" :c,
                "가격":p,
                "링크":u
            })
        for item in result:
            if item["이름"] not in result_list:
                result_list.append(item)

                
                
    #최저 가격 숙소 / 최고 평점 숙소 구하기            
    min_price=1000000
    name_of_min_price=""
    min_url=""
    
    highest_rate=0.00
    name_of_hr=""
    highest_url=""
    
    for item in result_list:
        price=item["가격"]
        price=price.replace("₩","")
        price=int(price.replace(",",""))

        if price<min_price:
            min_price=price
            name_of_min_price=item["이름"]
            min_url=item["링크"]
            
        rate=float(item["평점"])
        if rate>highest_rate:
            highest_rate=rate
            name_of_hr=item["이름"]
            highest_url=item["링크"]


    return result_list,"최저 가격 숙소 : %s (가격 : %d), (url : %s )" %(name_of_min_price,min_price,min_url),"최고 평점 숙소 : %s (평점 : %.2f), (url : %s)" %(name_of_hr,highest_rate,highest_url)
    


In [256]:
air=airbnb("제주","2020-08-16","2020-08-17",5)

In [257]:
air

([{'이름': '몽돌하우스',
   '평점': '4.84',
   '후기 개수': '63개',
   '가격': '34,543',
   '링크': 'https://www.airbnb.co.kr/rooms/15321610?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=320cafb3-27b8-430d-b74c-420e602b1fdb'},
  {'이름': '[플레이스캠프제주] Standard',
   '평점': '4.47',
   '후기 개수': '702개',
   '가격': '44,100',
   '링크': 'https://www.airbnb.co.kr/rooms/19724369?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=320cafb3-27b8-430d-b74c-420e602b1fdb'},
  {'이름': "#2 / JW's House 2 (독채)",
   '평점': '4.68',
   '후기 개수': '317개',
   '가격': '103,977',
   '링크': 'https://www.airbnb.co.kr/rooms/15639787?location=%EC%A0%9C%EC%A3%BC%EB%8F%84&check_in=2020-08-16&check_out=2020-08-17&previous_page_section_name=1000&federated_search_id=320cafb3-27b8-430d-b74c-420e602b1fdb'},
  {'이름': '걷기 좋은 동네 게스트하우스',
   '평점': '4.61',
   '후기 개수': '70개',
   '가격': '35,814',
   '

In [258]:
import pandas as pd

In [259]:
dt=pd.DataFrame(air[0])

In [261]:
dt.to_excel("airbnb_crawling.xlsx")